In [6]:
import utils
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

In [7]:
tr_df = pd.read_csv('data/adult.data', header=None)

In [8]:
columns = ["age", "workclass", "fnlwgt", "education", "education-num", 
           "marital-status", "occupation", "relationship", "race", "sex",
           "capital-gain","capital-loss","hours-per-week","native-country",
           "income"
          ]
index_c = {}
for i in range(len(columns)):
    index_c[i]=columns[i]
tr_df.rename(columns=index_c)
tr_df.columns = columns
tr_df.info()
tr_df
# categorize the data
# tr_df[1, 3,5,6,7,8,9,13,14] #= tr_df[1].astype('category')
# for i in [1, 3,5,6,7,8,9,13,14]:
#     tr_df[i] = tr_df[i].astype('category')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [9]:
tr_np = tr_df.to_numpy()

In [10]:
tr_df_i = tr_df.copy()


In [11]:
for i in columns:
    labelencoder= LabelEncoder()
    tr_df_i[i] = labelencoder.fit_transform(tr_df_i[i])

In [12]:
tr_df_i

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,22,7,2671,9,12,4,1,1,4,1,25,0,39,39,0
1,33,6,2926,9,12,2,4,0,4,1,0,0,12,39,0
2,21,4,14086,11,8,0,6,1,4,1,0,0,39,39,0
3,36,4,15336,1,6,2,6,0,2,1,0,0,39,39,0
4,11,4,19355,9,12,2,10,5,2,0,0,0,39,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,10,4,16528,7,11,2,13,5,4,0,0,0,37,39,0
32557,23,4,8080,11,8,2,7,0,4,1,0,0,39,39,1
32558,41,4,7883,11,8,6,1,4,4,0,0,0,39,39,0
32559,5,4,12881,11,8,4,1,3,4,1,0,0,19,39,0


In [13]:
tr_df_i.describe()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
count,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000
mean,21.580326,3.868892,10498.892479,10.298210,9.080679,2.611836,6.572740,1.446362,3.665858,0.669205,6.501213,2.053285,39.377937,36.718866,0.240810
std,13.635502,1.455960,6048.972814,3.870264,2.572720,1.506222,4.228857,1.606771,0.848806,0.470506,23.300287,10.056564,12.144006,7.823782,0.427581
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,11.000000,4.000000,5396.000000,9.000000,8.000000,2.000000,3.000000,0.000000,4.000000,0.000000,0.000000,0.000000,39.000000,39.000000,0.000000
50%,20.000000,4.000000,10348.000000,11.000000,9.000000,2.000000,7.000000,1.000000,4.000000,1.000000,0.000000,0.000000,39.000000,39.000000,0.000000
75%,31.000000,4.000000,15487.000000,12.000000,11.000000,4.000000,10.000000,3.000000,4.000000,1.000000,0.000000,0.000000,44.000000,39.000000,0.000000
max,72.000000,8.000000,21647.000000,15.000000,15.000000,6.000000,14.000000,5.000000,4.000000,1.000000,118.000000,91.000000,93.000000,41.000000,1.000000


In [14]:
data_np = tr_df_i.to_numpy(dtype='float')

In [15]:
data_np.shape
X, y = data_np[:,:-1], data_np[:,-1]

In [16]:
scaler = MinMaxScaler()
X_tr = scaler.fit_transform(X)

In [17]:
X_tr.shape, y.shape

((32561, 14), (32561,))

In [41]:
model = nn.Sequential(
    nn.Linear(14,1000),
    nn.ReLU(),
    nn.Linear(1000,6400),
    nn.ReLU(),
    nn.Linear(6400,3200),
    nn.ReLU(),
    nn.Linear(3200,200),
    nn.ReLU(),
    nn.Linear(200, 2)
)
criterion = nn.CrossEntropyLoss(reduction='sum')
batch_size = 64
# optimizer = torch.optim.Adam(model.parameters(), lr=3e-1)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
epochs = 10

In [42]:
## split to train and val
X_val, y_val = X_tr[-500:, :], y[-500:]
X_tr, y=X_tr[:-500, :], y[:-500]
N, D = X_tr.shape
print(N, D)

31061 14


In [43]:
writer = SummaryWriter(flush_secs=10)
loss_history=[]
for e in range(epochs):
    for i in range(0, N, batch_size):
        optimizer.zero_grad()
        X_batch = torch.as_tensor(X_tr[i:i+batch_size,:].reshape((-1,D)), dtype=torch.float)
        y_batch = torch.as_tensor(y[i:i+batch_size], dtype=torch.long)
        loss=criterion(model(X_batch), y_batch)
        loss.backward()
        optimizer.step()
        if writer is not None:
                writer.add_scalar("Loss/Train", loss.item())
        if i%10 == 9:
            loss_history.append(loss.item())
        if (i/batch_size)%200 == 99:
            print("epoch %i: iteration %i/%i= %s" % (e+1, i, N, str(loss.detach().numpy())))
            print('Accuracy: ', torch.sum(torch.argmax(model(torch.as_tensor(X_val, dtype=torch.float)), axis=1) == torch.as_tensor(y_val, dtype=torch.int))/float(len(y_val)))
            
print('Final Accuracy: ', torch.sum(torch.argmax(model(torch.as_tensor(X_val, dtype=torch.float)), axis=1) == torch.as_tensor(y_val, dtype=torch.int))/float(len(y_val)))

epoch 1: iteration 6336/31061= 31.62421
Accuracy:  tensor(0.7100)
epoch 1: iteration 19136/31061= 30.05383
Accuracy:  tensor(0.7780)
epoch 2: iteration 6336/31061= 21.41134
Accuracy:  tensor(0.8000)
epoch 2: iteration 19136/31061= 20.66731
Accuracy:  tensor(0.7740)
epoch 3: iteration 6336/31061= 22.426567
Accuracy:  tensor(0.8240)
epoch 3: iteration 19136/31061= 17.986664
Accuracy:  tensor(0.7560)


KeyboardInterrupt: 

In [40]:
X_val_t = torch.as_tensor(X_val)
X_val_t.requires_grad = True
test_crit = nn.

True
